## Preparando o Ambiente

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [13]:
import os
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from datasets import load_from_disk
from peft import LoraConfig, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)
from tqdm.notebook import tqdm

#from trl import SFTTrainer # Problema ao importar essa biblioteca

In [32]:
from trl import SFTTrainer

ImportError: cannot import name 'top_k_top_p_filtering' from 'transformers' (d:\PROJETOS LLM\Microsoft PHI_3 LLM\.venv\Lib\site-packages\transformers\__init__.py)

In [15]:
from huggingface_hub import interpreter_login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token can be pasted using 'Right-Click'.
Token is valid (permission: write).
Your token has been saved in your configu

## Criação do Dataset de Ajuste

In [19]:
import pandas as pd

# Dados em formato de dicionário
data = {
    "Instrução": [
        "Qual é a composição de Vi-Ferrin® solução oral gotas por mililitro?",
        "Como deve ser armazenado o Vi-Ferrin®?",
        "Para que condições médicas o Vi-Ferrin® é indicado?",
        "Vi-Ferrin® solução oral é indicado para quais grupos etários?",
        "Quais são as principais interações medicamentosas que afetam a absorção do ferro em Vi-Ferrin®?",
        "Quais precauções devem ser consideradas ao usar Vi-Ferrin® durante a gravidez?",
        "Como Vi-Ferrin® funciona no tratamento da anemia?",
        "Quais são as contraindicações para o uso de Vi-Ferrin®?",
        "Quais efeitos adversos são comuns com o uso de Vi-Ferrin®?",
        "O que fazer em caso de superdose de Vi-Ferrin®?",
        "Quais instruções de uso são específicas para Vi-Ferrin® solução oral gotas para lactentes?",
        "Como deve ser utilizado o Vi-Ferrin® solução oral para crianças acima de 20 kg?",
        "Existem recomendações específicas para a administração de Vi-Ferrin® para adultos?",
        "Como o Vi-Ferrin® deve ser administrado para crianças até 20 kg?",
        "Quais interações medicamento-substância química são importantes para usuários de Vi-Ferrin®?",
        "Como as interações medicamento-exame laboratorial afetam os resultados dos testes?",
        "Quais cuidados especiais são necessários para pacientes com condições particulares ao usar Vi-Ferrin®?",
        "Quais são as características organolépticas de Vi-Ferrin® solução oral?",
        "Como deve ser feita a administração de Vi-Ferrin® solução oral gotas?",
        "O que fazer se esquecer uma dose de Vi-Ferrin®?"
    ],
    "Saída": [
        "Cada 1 mL de Vi-Ferrin® solução oral gotas contém 150 mg de quelato de ferro, 0,25 mg de ácido fólico e 7,5 mcg de cianocobalamina, além de excipientes como sacarina, hidróxido de sódio, propilenoglicol, metilparabeno, propilparabeno, essência de laranja e água de osmose reversa.",
        "Vi-Ferrin® deve ser conservado em temperatura ambiente, entre 15°C e 30ºC, protegido da luz e umidade.",
        "Vi-Ferrin® é indicado para o tratamento de anemias por deficiência de ferro, estados de desnutrição e convalescença.",
        "Vi-Ferrin® solução oral é indicado para uso adulto e pediátrico.",
        "Medicamentos como fluoroquinolonas (norfloxacino e ciprofloxacino), tetraciclinas (oxitetraciclina e doxiciclina), fenitoína, antiácidos (hidróxido de alumínio e magnésio) e colestiramina podem diminuir a absorção do ferro. O ácido ascórbico, por outro lado, aumenta a absorção do ferro, mas também aumenta os efeitos adversos.",
        "Vi-Ferrin® não deve ser utilizado por mulheres grávidas sem orientação médica ou do cirurgião-dentista.",
        "Vi-Ferrin® contém ferro, que é essencial para combater a anemia. O ácido fólico presente ajuda na transformação e composição de aminoácidos, enquanto a cianocobalamina desempenha um papel crucial na formação do DNA.",
        "Vi-Ferrin® não deve ser usado em casos de hipersensibilidade aos componentes da fórmula, em situações de transfusões de sangue repetidas ou em casos de anemia não causada por deficiência de ferro.",
        "Os efeitos adversos comuns incluem dor abdominal, constipação, náusea, vômito, e fezes escuras. Em doses elevadas, o uso oral de ácido fólico pode causar coloração amarela na urina.",
        "Em caso de superdose, os sintomas incluem dor abdominal, diarreia, vômitos com conteúdo gástrico marrom ou sanguinolento, e sintomas mais graves como palidez, cansaço, e sonolência. Deve ser procurado socorro médico imediatamente e, se possível, induzir o vômito se a ingestão for recente.",
        "Para lactentes e crianças de até 1 ano de idade, deve-se administrar 1 gota (0,05 mL) por kg de peso ao dia de Vi-Ferrin® solução oral gotas.",
        "Crianças acima de 20 kg devem tomar 15 mL de Vi-Ferrin® solução oral uma vez ao dia, preferencialmente antes da principal refeição.",
        "Adultos devem tomar 15 mL de Vi-Ferrin® solução oral, duas vezes ao dia, preferencialmente antes das refeições.",
        "Crianças até 20 kg devem tomar 5 mL de Vi-Ferrin® solução oral, duas vezes ao dia, preferencialmente antes das refeições.",
        "O consumo de bebidas alcoólicas pode prejudicar a eficácia do tratamento com Vi-Ferrin®, afetando a absorção de seus componentes.",
        "Antibióticos podem interferir com ensaios microbiológicos para determinar as concentrações de ácido fólico, produzindo resultados falsamente baixos. A administração de ácido fólico pode normalizar a contagem sanguínea em pacientes com deficiência de vitamina B12, mascarando a melhora da anemia.",
        "Pacientes com anemia perniciosa, artrite reumatoide, insuficiência renal crônica, redução da função hepática, doença de Hodgkin, úlcera péptica, enterite regional, colite ulcerativa, estreitamento intestinal ou diverticulose devem consultar um médico antes de usar Vi-Ferrin® devido ao risco de problemas neurológicos e outras complicações.",
        "Vi-Ferrin® solução oral é um líquido límpido de cor vermelho escuro a castanho avermelhado com odor de laranja.",
        "Para administrar Vi-Ferrin® solução oral gotas, o frasco deve ser virado e mantido na posição vertical. Para iniciar o gotejamento, bater levemente com o dedo no fundo do frasco.",
        "Se esquecer de tomar uma dose de Vi-Ferrin®, continue o tratamento como recomendado, pulando a dose esquecida. Não duplique a dose seguinte."
    ]
}

# Criando o DataFrame
df = pd.DataFrame(data)

# Mostrando o DataFrame
df.head(2)

,Instrução,Saída
0,Qual é a composição de Vi-Ferrin® solução oral...,Cada 1 mL de Vi-Ferrin® solução oral gotas con...
1,Como deve ser armazenado o Vi-Ferrin®?,Vi-Ferrin® deve ser conservado em temperatura ...


In [20]:
df.columns

Index(['Instrução', 'Saída'], dtype='object')

In [21]:
# Function to transform the row into desired format
def format_row(row):
    question = row['Instrução']
    answer = row['Saída']
    formatted_string = f"[INST] {question} [/INST] {answer} "
    return formatted_string

# Apply the function to each row of the dataframe
df['Formatted'] = df.apply(format_row, axis=1)

# Display the formatted column
df['Formatted']

0     [INST] Qual é a composição de Vi-Ferrin® soluç...
1     [INST] Como deve ser armazenado o Vi-Ferrin®? ...
2     [INST] Para que condições médicas o Vi-Ferrin®...
3     [INST] Vi-Ferrin® solução oral é indicado para...
4     [INST] Quais são as principais interações medi...
5     [INST] Quais precauções devem ser consideradas...
6     [INST] Como Vi-Ferrin® funciona no tratamento ...
7     [INST] Quais são as contraindicações para o us...
8     [INST] Quais efeitos adversos são comuns com o...
9     [INST] O que fazer em caso de superdose de Vi-...
10    [INST] Quais instruções de uso são específicas...
11    [INST] Como deve ser utilizado o Vi-Ferrin® so...
12    [INST] Existem recomendações específicas para ...
13    [INST] Como o Vi-Ferrin® deve ser administrado...
14    [INST] Quais interações medicamento-substância...
15    [INST] Como as interações medicamento-exame la...
16    [INST] Quais cuidados especiais são necessário...
17    [INST] Quais são as características organo

In [22]:
# Rename the 'Formatted' column to 'Text'
new_df = df.rename(columns={'Formatted': 'Text'})

new_df

,Instrução,Saída,Text
0,Qual é a composição de Vi-Ferrin® solução oral...,Cada 1 mL de Vi-Ferrin® solução oral gotas con...,[INST] Qual é a composição de Vi-Ferrin® soluç...
1,Como deve ser armazenado o Vi-Ferrin®?,Vi-Ferrin® deve ser conservado em temperatura ...,[INST] Como deve ser armazenado o Vi-Ferrin®? ...
2,Para que condições médicas o Vi-Ferrin® é indi...,Vi-Ferrin® é indicado para o tratamento de ane...,[INST] Para que condições médicas o Vi-Ferrin®...
3,Vi-Ferrin® solução oral é indicado para quais ...,Vi-Ferrin® solução oral é indicado para uso ad...,[INST] Vi-Ferrin® solução oral é indicado para...
4,Quais são as principais interações medicamento...,Medicamentos como fluoroquinolonas (norfloxaci...,[INST] Quais são as principais interações medi...
5,Quais precauções devem ser consideradas ao usa...,Vi-Ferrin® não deve ser utilizado por mulheres...,[INST] Quais precauções devem ser consideradas...
6,Como Vi-Ferrin® funciona no tratamento da anemia?,"Vi-Ferrin® contém ferro, que é essencial para ...",[INST] Como Vi-Ferrin® funciona no tratamento ...
7,Quais são as contraindicações para o uso de Vi...,Vi-Ferrin® não deve ser usado em casos de hipe...,[INST] Quais são as contraindicações para o us...
8,Quais efeitos adversos são comuns com o uso de...,Os efeitos adversos comuns incluem dor abdomin...,[INST] Quais efeitos adversos são comuns com o...
9,O que fazer em caso de superdose de Vi-Ferrin®?,"Em caso de superdose, os sintomas incluem dor ...",[INST] O que fazer em caso de superdose de Vi-...


In [23]:
new_df = new_df[['Text']]
new_df.head(5)

,Text
0,[INST] Qual é a composição de Vi-Ferrin® soluç...
1,[INST] Como deve ser armazenado o Vi-Ferrin®? ...
2,[INST] Para que condições médicas o Vi-Ferrin®...
3,[INST] Vi-Ferrin® solução oral é indicado para...
4,[INST] Quais são as principais interações medi...


In [24]:
# If you want to save the new dataframe to a CSV file:
new_df.to_csv('formatted_data.csv', index=False)
final_df = pd.read_csv("formatted_data.csv")
final_df.head(3)

,Text
0,[INST] Qual é a composição de Vi-Ferrin® soluç...
1,[INST] Como deve ser armazenado o Vi-Ferrin®? ...
2,[INST] Para que condições médicas o Vi-Ferrin®...


In [25]:
training_dataset = load_dataset("csv", data_files="formatted_data.csv", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [26]:
training_dataset

Dataset({
    features: ['Text'],
    num_rows: 20
})

## Ajuste do modelo

In [31]:
base_model = "microsoft/phi-2"
new_model = "phi-2-test-pharma"

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    # use_flash_attention_2=True, # Phi does not support yet.
    trust_remote_code=True,
    flash_attn=True,
    flash_rotary=True,
    fused_dense=True,
    low_cpu_mem_usage=True,
    device_map={"": 0},
    revision="refs/pr/23",
)

model.config.use_cache = False
model.config.pretraining_tp = 1

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=32,
    evaluation_strategy="steps",
    eval_steps=2000,
    logging_steps=15,
    optim="paged_adamw_8bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_steps=2000,
    warmup_ratio=0.05,
    weight_decay=0.01,
    max_steps=-1
)

peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules= ["Wqkv", "fc1", "fc2" ] # ["Wqkv", "out_proj", "fc1", "fc2" ], - 41M params
    # modules_to_save=["embed_tokens","lm_head"]
)

trainer = SFTTrainer(
    model=model,
    train_dataset=training_dataset,
    peft_config=peft_config,
    dataset_text_field="Text",
    max_seq_length=690,
    tokenizer=tokenizer,
    args=training_arguments,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`